In [ ]:
from groq import Groq
from tqdm import tqdm
import pandas as pd
import ujson
import time
import json
import ast
import re

In [ ]:
submission, claims, retrievals = [], [], []

file_path1 = '../data/test.json'
file_path2 = '../data/retrieval.json'

with open(file_path1, 'r') as file:
    data1 = json.load(file)

with open(file_path2, 'r') as file:
    data2 = json.load(file)

for item in data1:
    dic = {"claim_id": item.get("claim_id"), "claim": item.get("claim"), "pred_label": "", "evidence": []}
    submission.append(dic)
    claims.append(item.get("claim"))

for item in data2:
    retrievals.append(item.get("retrievals"))

In [ ]:
api_key=""

def get_model_response(prompt):
    system_message = {
        'role': 'system',
        'content': "You are a helpful assistant with expertise in creating evidence through suitable question-answer pairs based on a given claim and the available key points within the retrieved knowledge, and in providing an accurate verdict for that claim."
    }
    user_message = {
        'role': 'user',
        'content': prompt
    }

    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[system_message, user_message],
        temperature=0,
        top_p = 0.8,
        max_tokens=1024,
        seed = 42
    )

    response = completion.choices[0].message.content
    return response

folder_path = '../knowledge_store/'
counter = 0

for i in tqdm(range(len(claims))):
    client = Groq(api_key=api_key)
    claim = claims[i]
    retrieval = retrievals[i]
    prompt = """Your task is to accurately determine a correct verdict for a given claim from the labels "Refuted", "Supported", "Not Enough Evidence", or "Conflicting Evidence/Cherry-Picking". You need to provide 1 to 4 necessary and helpful question-answer (QA) pairs. Each QA pair should be well-constructed, focusing on different important parts of the claim and utilizing the retrieved knowledge effectively to guide accurate decision-making. Therefore, you need to break down the claim into its distinct and most important subclaims, focusing on these individual components, as well as considering direct questions related to the main claim if the retrieved knowledge is sufficient. Your answers can only be in the forms of extractive (preferred), abstractive, or unanswerable. Extractive answers are those directly pulled from the text, while abstractive answers summarize or infer information based on the text. Unanswerable type is very rare, and in this case, set the answer to "No answer could be found." and the citation_id to "". Each piece of text in the retrieved knowledge has a <citation_id> at its end, where the placeholder is replaced by the main citation ID. For each proposed answer to all answerable questions in your evidence, you must include exactly one citation ID (if there are multiple citation_id, select only one) solely within the "citation_id" field. After providing evidence, you must also provide a concise justification explaining how the evidence and the retrieved knowledge support the selected label for the claim. Provide your answer explicitly in the following format without any other change or additional feedback:\n{\n  "evidence": [\n    {\n      "question": "question",\n      "answer": "answer",\n      "citation_id": "<citation_id>"\n    },\n    ...\n  ],\n  "justification": "justification",\n  "pred_label": "pred_label"\n}\n\n"""+f"""Claim:\n{claim}\n\nRetrieved Knowledge:\n{retrieval}"""
    
    try:
        response = get_model_response(prompt)
    except Exception as e:
        time.sleep(5)
        try:
            response = get_model_response(prompt)
        except Exception as e:
            time.sleep(5)
            try:
                response = get_model_response(prompt)
            except Exception as e:
                counter += 1
                continue

    start_idx = str(response).find('{')
    end_idx = str(response).rfind('}') + 1

    if start_idx != -1 and end_idx != -1:
        try:
            response = ast.literal_eval(response[start_idx:end_idx])
        except Exception as e:
            counter += 1
            continue
    else:
        counter += 1
        continue

    submission[i]["pred_label"] = response["pred_label"]
    for item in response["evidence"]:
        dic = {"question": item["question"], "answer": item["answer"], "url": "", "scraped_text": ""}
        if item["citation_id"] != "":
            if "<" in item["citation_id"]:
                item["citation_id"] = item["citation_id"].replace("<", "")
                item["citation_id"] = item["citation_id"].replace(">", "")
            if "," in item["citation_id"]:
                item["citation_id"] = item["citation_id"].split(",")[0].strip()
            url_index = int(item["citation_id"].split("_")[0])
            scraped_text_index = int(item["citation_id"].split("_")[1])
            file_path = folder_path + f"{i}.json"
            with open(file_path, 'r', encoding='utf-8') as file:
                for current_index, line in enumerate(file):
                    if current_index == url_index:
                        row = ujson.loads(line)
                        break
            dic["url"] = row["url"]
            dic["scraped_text"] = row["url2text"][scraped_text_index]
        submission[i]["evidence"].append(dic)

    counter += 1

with open('../data/predictions.json', 'w') as f:
    json.dump(submission, f, indent=4)